# Text Generation with Markov Chains and Neural Networks

## Introduction

Today we'll focus on a few different methods for generating text. Text generation is a challenging problem that even the largest data science teams are still struggling with, so we'll explore some of the most common and accessible methods to solve the problem, starting at a somewhat basic level. The approaches we'll attempt are:

* Markov Chains
* Recurrent Neural Networks/Long-Short Term Memory Networks (an extension of RNNs)

For the later methods, we'll need Keras. If you haven't already installed Keras, you can use:

`conda install -c conda-forge keras tensorflow`

or in Windows, by using the Anaconda Navigator to install the keras and tensorflow packages.



## Question 1

* Create a class called markov_chain that takes the path of a text file as input. Add a function that preprocesses the text returning a list of lowercased words with all " and ' removed.
* Now add a function that reads the text and creates a dictionary of the form: `{(word1, word2): [word3], (word2, word3): [word4]...}`. If the sequence `(word1, word2, word4)` then shows up later.. we want to end up with `{(word1, word2): [word3, word4], (word2, word3): [word4]...}`. We want to map out how often every word follows each previous pair of words. If `(word1, word2, word3)` appears a second time then we want `{(word1, word2): [word3, word4, word3], (word2, word3): [word4]...}`.
* Add a function for generating new text from a seed, that takes in a key (e.g. `(word1, word2)`) as a starting point, then randomly samples one of the words that follows that key. (e.g. word3). Then have that sampled word appended to the "generated words" creating a new key (e.g. `(word2, word3)`). Repeat this process over and over to generate a sentence until reaching some sentence length as specified by the user. Return this sentence as a string.
* Modify your code to allow more than 2 words in a key. Make that a parameter the user can set (for instance markov_chain(PATH, ngram = 3) would have a dictionary with a format `{(word1, word2, word3): [word4]}`. Does adding more ngrams make the sentences more intelligible? What is the downside to having large keys in a Markov Generator?

In [ ]:
import random
class markov_chain(object):
    
    def __init__(self,text_path,ngram=2):
        self.ngram = ngram
        self.markov_keys = dict()
        self.path = text_path
        self.text_as_list = None

    def preprocess(self):
        with open(self.path,'r') as f:
            raw = f.read()
        self.text_as_list = # student section here

    def markov_group_generator(self,text_as_list):
        if len(text_as_list) < self.ngram+1:
            raise("NOT A LONG ENOUGH TEXT!")
            return

        for i in range(self.ngram,len(text_as_list)):
            yield tuple(text_as_list[i-self.ngram:i+1])

    def create_probability_object(self):
        # student section here
        
        return None
    
    def generate_sentence(self, length=25, starting_word_id=None):
        # student section here
        
        return None
        
        

In [ ]:
MC = markov_chain('../data/lovecraft.txt',ngram=2)

In [ ]:
MC.create_probability_object()

In [ ]:
i = 0
for key,value in MC.markov_keys.items():
    print(key,value)
    print()
    i+=1
    if i>50:
        break

In [ ]:
print(MC.generate_sentence(length=100, starting_word_id=25))

Adding more keys in general does make the sentences better... because we have more and more words that must be in a specific order. Eventually, if you increase the key size large enough, you take out most of the randomization because there will only be a few times that 25 words appear in a certain order. You'll end up just writing the input text back again.

## Question 2

Now let's switch gears to do character level text generation using neural nets. We want to frame this as a classification question: "given this sequence of characters, which type of character should come next?" Example: Given "the quick brown fo" we want to be able to predict that the next character should be "x".

* Read in the text from: https://s3.amazonaws.com/text-datasets/nietzsche.txt (stored in week08/data/nietzsche.txt). Create a map that assigns every unique character in the text an ID number. Also create one that maps from ID back to character as well.
* Convert the text into lists of characters (with some max length) for input and an associated output. For example: if maxlength = 5 and we had the sentence: "the quick brown fox", we'd want to split into `X = [['t','h','e',' ','q'],['h','e',' ','q','u']...]` and `y = ['u','i',...]`. Each of these sequences will act as an record-label pair for training our network.
* For each sequence, encode the input-output couple as a set of binary arrays. The inputs and output will both be arrays with length equal to the number of unique characters, but only those markers at the positions of IDs that are in the sequence should be non-zero. As an example, "the quick brown fox" has 16 unique characters: 
    
    `['t', 'h', 'e', ' ', 'q', 'u', 'i', 'c', 'k', 'b', 'r', 'o', 'w', 'n', 'f', 'o', 'x']`. 
    
    So an array set for an input sequence `qui` would be (if maxlength = 3):
    
    `[[0, 0, 0, 0, 1, 0, 0, 0, 0, ...],[0, 0, 0, 0, 0, 1, 0, 0, 0, ...],[0, 0, 0, 0, 0, 0, 1, 0, 0, ...]].`
    
    And the output for `c` would be:
    
    `[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0...].`
    
    You'll need the Character-to-ID map we created earlier to do this. The reason the input becomes 3 inputs is that LSTMs want to know about the **arrangement** of the characters... so it matters that 'q' comes before 'u' which comes before 'i' in this case. 
* Use a sequence length of 10 and generate both the X and y (sequence and associated label) to be used in the LSTM. The choice of 10 is to speed up training, if you have the time, training with a sequence length of 40 does a better job overall.

In [ ]:
import numpy as np 

path = '../data/nietzsche.txt'
text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = None # student section here
indices_char = None # student section here
MAX = 40
maxlen=MAX

# cut the text in semi-redundant sequences of maxlen characters
def chop_into_sequences(text, char_indices, indices_char, maxlen=20):
    sentences = []
    next_chars = []
    # student section here
    
    print('nb sequences:', len(sentences))

    print('Vectorization...')
    X = None # student section here
    y = None # student section here
    for i, sentence in enumerate(sentences):
        for t, char in enumerate(sentence):
            X[i, t, char_indices[char]] = 1
        y[i, char_indices[next_chars[i]]] = 1
    return X, y

X,y = chop_into_sequences(text, char_indices, indices_char, maxlen=3)

In [ ]:
print(X[0])

In [ ]:
X,y = chop_into_sequences(text, char_indices, indices_char, maxlen=MAX)

## Question 3

* Create a sequential model with Keras (https://keras.io/models/sequential/)
* Add one LSTM layer with 128 nodes, with an input shape that's equal to (number of characters to include, number of unique characters)
* Add a Dense layer of the shape (number of unique characters)
* Add a final 'softmax' activation layer. Then compile the model.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, Reshape, Activation, SimpleRNN, GRU, LSTM, Convolution1D, MaxPooling1D, Merge, Dropout
from keras.optimizers import SGD, RMSprop

In [ ]:
# build the model: a single LSTM



## Question 4

* With the model ready, fit it with the sampled X and y's from above. Run it through a few epochs with a batch size of ~128. (This may take a while, so start with just a few epochs).
* After the fit, choose a starting seed from the corpus, and force it into the correct shape for the model to predict from (same shape as X above). Do a prediction of the next character, then remove the first character and add the new character and repeat the process 400 times, generating sentences.
* I've provided a few pre-trained sets of weights along with the exercise for the Nietzsche dataset, with max length 10 and 40. If you are having trouble getting the model to train, you can also use the "load_weights" method of a sequential model, as long as you've properly followed the instructions above and given it the same number of nodes, max length, and character sets as I have. If your training keeps breaking or is taking too long, you can try to load the weights to practice generation.

In [ ]:
import sys

def sample(preds, temperature=None):
    # helper function to sample an index from a probability array
    
    # If the user specifies a temperature, we use this to and some more randomization
    if temperature:
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)
    # if not, we just return the most likely prediction. This is likely to get trapped in loops.
    else:
        return np.argmax(preds)

In [ ]:
# train the model, output generated text after each iteration
try:
    model.load_weights("LSTM_128_"+str(MAX)+"_Nietzsche.h5")
    print("LOADED PREVIOUS WEIGHTS FOR THIS MAX LENGTH.")
except:
    print("NO PREVIOUS WEIGHTS FOR THIS LENGTH.")
    
for iteration in range(1, 4):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(X, y, batch_size=128, nb_epoch=1)
    model.save_weights("LSTM_128_"+str(MAX)+"_Nietzsche.h5")

    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        generated = ''
        sentence = text[start_index: start_index + maxlen]
        generated += sentence
        print('----- Generating with seed: "' + sentence + '" -----')
        sys.stdout.write(generated)

        for i in range(400):
            x = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
        print(generated)

This can be used to generate without a fit, as long as the weight files are loaded properly.

In [ ]:
import sys
import random

start_index = random.randint(0, len(text) - maxlen - 1)

# student section here


# student section ends here    
model.load_weights("LSTM_128_"+str(MAX)+"_Nietzsche.h5")

generated = ''
sentence = text[start_index: start_index + maxlen]
generated += sentence
print('----- Generating with seed: "' + sentence + '"')
sys.stdout.write(generated)

for i in range(400):
    x = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x[0, t, char_indices[char]] = 1.

    preds = model.predict(x, verbose=0)[0]
    next_index = sample(preds, temperature=0.2)
    next_char = indices_char[next_index]

    generated += next_char
    sentence = sentence[1:] + next_char

    #sys.stdout.write(next_char)
    #sys.stdout.flush()
print(generated)

This clearly needs more training, as it's getting stuck in a lot of "the so's" and whatnot. However, we're already getting a lot of recognizable words. Remember, we never actually told it the word "dance" is a word. We told it that the letters 'd', 'a', 'n', 'c', and 'e' do sometimes appear around each other though. So it's learning a bit about the English language, from a spelling/phonetic point of view. With more training, it will likely coalesce some more into a more accurate representation with less repetition and fewer spelling errors. It's not perfect though, and it will never truly understand the sentences it's making. It's a great start for text generation though and is fairly state of the art at the moment.

In [ ]:
generated